# 🔐 Flujo de Trabajo: Tests Privados con SSH Keys

Este notebook contiene el flujo completo para configurar tests privados usando claves SSH y repositorios privados en GitHub Actions.

## 📋 **Paso 1: Generar claves SSH (PowerShell)**

Ejecuta estos comandos en PowerShell (no en este notebook):

In [ ]:
# Generar par de claves SSH
ssh-keygen -t ed25519 -C "github-actions"

# Presiona Enter en cada pregunta:
# - Ruta por defecto (Enter)
# - Sin contraseña (Enter)
# - Confirmar sin contraseña (Enter)

In [ ]:
# Ver la clave pública (para GitHub)
Get-Content $env:USERPROFILE\.ssh\id_ed25519.pub

# Ver la clave privada (para secret)
Get-Content $env:USERPROFILE\.ssh\id_ed25519

## 🔑 **Paso 2: Configurar claves en GitHub**

### **2.1 Agregar clave pública a tu cuenta:**

1. Ve a GitHub.com → **Settings** → **SSH and GPG keys** → **New SSH key**
2. **Title:** "Taller Actions Key" (o cualquier nombre)
3. **Key type:** Authentication Key
4. **Key:** Pega el contenido de `id_ed25519.pub`
5. **Add SSH key**

### **2.2 Agregar clave privada como secret:**

1. Ve a tu repositorio → **Settings** → **Secrets and variables** → **Actions**
2. **New repository secret**
3. **Name:** `ACTIONS_SSH_KEY` (o cualquier nombre)
4. **Secret:** Pega el contenido completo de `id_ed25519` (desde `-----BEGIN` hasta `-----END`)
5. **Add secret**

## 📁 **Paso 3: Organizar tests en repositorio privado**

Estructura recomendada en el repositorio privado:

In [ ]:
Repositorio privado: Tests/
├── test_validacion/
│   ├── ejercicio1/
│   │   ├── test_1.py (validación de no modificación)
│   │   └── test_2.py (anti-cheating avanzado)
│   ├── ejercicio2/
│   │   ├── test_4.py
│   │   └── test_5.py
│   ├── ejercicio3/
│   │   ├── test_6.py
│   │   └── test_7.py
│   └── ejercicio4/
│       ├── test_8.py
│       └── test_9.py

## ⚙️ **Paso 4: Configurar workflow (YAML)**

Contenido completo para `.github/workflows/python-tests.yml`:

In [ ]:
name: Taller NLP Tests

on: [push, pull_request]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v4
    
    - name: Set up Python
      uses: actions/setup-python@v5
      with:
        python-version: '3.10'
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
        pip install jupyter nbconvert ipykernel
        python -m ipykernel install --user --name=python3

    - name: Configurar clave SSH para git
      run: |
        mkdir -p ~/.ssh
        echo "${{ secrets.ACTIONS_SSH_KEY }}" > ~/.ssh/id_ed25519
        chmod 600 ~/.ssh/id_ed25519
        ssh-keyscan github.com >> ~/.ssh/known_hosts

    - name: Descargar tests privados desde repo privado
      run: |
        git clone git@github.com:Procesamiento-del-lenguaje-natural/Tests.git temp_repo
        cp temp_repo/test_validacion/ejercicio1/test_2.py tests/ejercicio1/test_2.py
        cp temp_repo/test_validacion/ejercicio2/test_4.py tests/ejercicio2/test_4.py
        cp temp_repo/test_validacion/ejercicio3/test_6.py tests/ejercicio3/test_6.py
        cp temp_repo/test_validacion/ejercicio4/test_8.py tests/ejercicio4/test_8.py
        rm -rf temp_repo

    - name: Ejecutar notebook ejercicio1
      run: |
        jupyter nbconvert --to notebook --execute notebooks/ejercicio1.ipynb --output ejercicio1_out.ipynb --ExecutePreprocessor.kernel_name=python3 --ExecutePreprocessor.timeout=300
      continue-on-error: true

    - name: Run tests ejercicio1 (privados)
      run: |
        pytest tests/ejercicio1/test_2.py -v
      continue-on-error: true

    - name: Ejecutar notebook ejercicio2
      run: |
        jupyter nbconvert --to notebook --execute notebooks/ejercicio2.ipynb --output ejercicio2_out.ipynb --ExecutePreprocessor.kernel_name=python3 --ExecutePreprocessor.timeout=300
      continue-on-error: true

    - name: Run tests ejercicio2 (privados)
      run: |
        pytest tests/ejercicio2/test_4.py -v
      continue-on-error: true

    - name: Ejecutar notebook ejercicio3
      run: |
        jupyter nbconvert --to notebook --execute notebooks/ejercicio3.ipynb --output ejercicio3_out.ipynb --ExecutePreprocessor.kernel_name=python3 --ExecutePreprocessor.timeout=300
      continue-on-error: true

    - name: Run tests ejercicio3 (privados)
      run: |
        pytest tests/ejercicio3/test_6.py -v
      continue-on-error: true

    - name: Ejecutar notebook ejercicio4
      run: |
        jupyter nbconvert --to notebook --execute notebooks/ejercicio4.ipynb --output ejercicio4_out.ipynb --ExecutePreprocessor.kernel_name=python3 --ExecutePreprocessor.timeout=300
      continue-on-error: true

    - name: Run tests ejercicio4 (privados)
      run: |
        pytest tests/ejercicio4/test_8.py -v
      continue-on-error: true

## ✅ **Paso 5: Verificación**

1. **Push tu workflow actualizado**
2. **Ve a Actions** en tu repositorio para ver la ejecución
3. **Verifica que:**
   - ✅ SSH se configura correctamente
   - ✅ El repo privado se clona sin errores
   - ✅ Los tests privados se copian y ejecutan
   - ✅ Los estudiantes ven los resultados pero no el código de los tests

## 🎯 **Resultado final:**

- **Estudiantes:** Ven resultados de tests, pueden debuggear con los errores mostrados
- **Profesor:** Tests completamente privados, imposibles de ver por estudiantes
- **Mantenimiento:** Un solo repositorio privado, una sola clave SSH para todo

## 📝 **Notas importantes:** 

1. **Secret name:** Cambia `ACTIONS_SSH_KEY` por el nombre que usaste al crear el secret
2. **Repo URL:** Cambia la URL del repositorio privado por la tuya
3. **Paths:** Ajusta las rutas de los tests según tu estructura
4. **Tests:** Agrega o quita tests según necesites

### **Para agregar más tests:**
Solo agrega líneas como:
```yaml
cp temp_repo/test_validacion/ejercicioX/test_Y.py tests/ejercicioX/test_Y.py
```

## 🔧 **Comandos útiles de troubleshooting:**

In [ ]:
# Verificar que las claves existen
Test-Path $env:USERPROFILE\.ssh\id_ed25519
Test-Path $env:USERPROFILE\.ssh\id_ed25519.pub

# Listar archivos en .ssh
Get-ChildItem $env:USERPROFILE\.ssh

# Probar conexión SSH a GitHub
ssh -T git@github.com